In [141]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn import svm
import matplotlib.pyplot as plt
from IPython.display import display


In [366]:
df_ohe = pd.read_csv('./Data/Final.csv')
# untested = pd.read_csv('./Data/Untested.csv')
df_ohe2 = df_ohe.copy()
# df_ohe = df_ohe.sample(frac=1).reset_index(drop=True)
# df_ohe2.tail(10)

In [145]:
# untested.podium = untested.podium.map(lambda x: 1 if x == 1 else 0)

Neutral network classifer (Winner)

In [379]:
df_ohe.podium = df_ohe.podium.map(lambda x: 1 if x == 1 else 0)



# train = df_ohe[(df_ohe['season'] <= 2021) | ((df_ohe['season'] == 2022) & (df_ohe['round'] <= 11))]
# test = df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] >= 12)]
train = df_ohe[(df_ohe['season'] <= 2021)]
test = df_ohe[(df_ohe['season'] == 2022)]
x_train = train.drop(['driver', 'podium', 'points'], axis = 1)
# x_test = test.drop(['driver', 'podium'], axis = 1)
y_train = train.podium
# y_test = test.podium

scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

In [385]:
final = []
#  & (df_ohe['round'] >= 12)
def score_classification(model):
    score = 0
    # final = []
    for circuit in df_ohe[(df_ohe.season == 2022)]['round'].unique():

        test = df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] == circuit)]
        test.reset_index(drop=True, inplace=True)

        # print(test.index)
        X_test = test.drop(['driver', 'podium', 'points'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df["actual"] = y_test
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df['proba_1']
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == max(prediction_df['proba_1'].values) else 0)
        predWinner = prediction_df[prediction_df['predicted'] == 1].first_valid_index()
        predWinnerName = test['driver'].loc[predWinner]
        # winnerName = winner['driver']
        # print(winner)
        # actualWinner = test[test['podium'] == 1].first_valid_index
        actualWinnerName = test.loc[test['podium'] == 1, 'driver'].iloc[0]#test.loc[test['podium'] == 1, 'driver'].first_valid_index
        # print(actualWinnerName)
        final.append((circuit, actualWinnerName, predWinnerName))
        # print(precision_score(prediction_df.actual, prediction_df.predicted))
        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] >= 12)]['round'].unique().max()
    return model_score, prediction_df, final

In [267]:
print(df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] >= 12)]['round'].unique().max())

22


In [356]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}


In [300]:
params={'hidden_layer_sizes': [(80,20,40,5), (75,25,50,10)], 
        'activation': ['identity', 'logistic', 'tanh', 'relu'], 
        'solver': ['lbfgs', 'sgd', 'adam', 'logistic'], 
        'alpha': np.logspace(-4,2,20)} 

for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        # for solver in params['solver']:
        for alpha in params['alpha']:
            model_params = (hidden_layer_sizes, activation, 'lbfgs', alpha )
            model = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes,
                                  activation = activation, solver = 'lbfgs', alpha = alpha, max_iter=10000, random_state = 1)
            model.fit(x_train, y_train)
            model_score = score_classification(model)
            comparison_dict['model'].append('neural_network_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [ ]:
comparison_df = pd.DataFrame(comparison_dict)

In [301]:
pd.DataFrame(comparison_dict).groupby('model')['params'].max()

model
neural_network_classifier    ((80, 20, 40, 5), tanh, lbfgs, 100.0)
Name: params, dtype: object

In [383]:
model = MLPClassifier(hidden_layer_sizes = (80, 20, 40, 5),
                                  activation = 'tanh', solver = 'lbfgs', alpha = 0.1, max_iter=10000, random_state = 1)

model.fit(x_train, y_train)

MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=(80, 20, 40, 5),
              max_iter=10000, random_state=1, solver='lbfgs')

In [381]:
model = MLPClassifier(hidden_layer_sizes = (75, 25, 50, 10),
                                  activation = 'identity', solver = 'lbfgs', alpha = 0.01623776739188721, max_iter=2000, random_state = 1)

model.fit(x_train, y_train)

MLPClassifier(activation='identity', alpha=0.01623776739188721,
              hidden_layer_sizes=(75, 25, 50, 10), max_iter=2000,
              random_state=1, solver='lbfgs')

In [346]:
model = MLPClassifier(hidden_layer_sizes = (100, 50, 25),
                                  activation = 'relu', solver = 'adam', alpha = 0.0001, learning_rate_init=0.001, max_iter=5000, random_state = 1)

model.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=5000, random_state=1)

In [386]:
score_classification(model)      

(0.2727272727272727,
      proba_0   proba_1  actual  predicted
 0   0.999876  0.000124       1          0
 1   0.999981  0.000019       0          0
 2   0.999983  0.000017       0          0
 3   0.999984  0.000016       0          0
 4   0.017113  0.982887       0          1
 5   0.999984  0.000016       0          0
 6   0.999984  0.000016       0          0
 7   0.999984  0.000016       0          0
 8   0.999983  0.000017       0          0
 9   0.999984  0.000016       0          0
 10  0.999984  0.000016       0          0
 11  0.999984  0.000016       0          0
 12  0.999984  0.000016       0          0
 13  0.999984  0.000016       0          0
 14  0.999984  0.000016       0          0
 15  0.999984  0.000016       0          0
 16  0.999984  0.000016       0          0
 17  0.999972  0.000028       0          0
 18  0.999984  0.000016       0          0
 19  0.999984  0.000016       0          0,
 [(1, 'leclerc', 'hamilton'),
  (2, 'max_verstappen', 'russell'),
  (3, 'le

In [170]:
# test_score = model.score(x_test, y_test)
# print("Test set score: {:.2f}".format(test_score))

In [389]:
tempFinal = pd.read_csv('Data/TempFinal.csv')

In [390]:
predictedDf = pd.DataFrame(final, columns=['CircuitId', 'Actual winner', 'Predicted Winner'])
circuit_name = []
i = 1
for value in predictedDf['CircuitId']:
    circuit_name.insert(i, tempFinal.loc[(tempFinal['round'] == value) & (tempFinal['season'] == 2022), 'circuit_id'].iloc[0])
    i += 1

predictedDf['Circuit Name'] = circuit_name

In [391]:
predictedDf.tail()
display(predictedDf)

,CircuitId,Actual winner,Predicted Winner,Circuit Name
0,1,leclerc,hamilton,bahrain
1,2,max_verstappen,russell,jeddah
2,3,leclerc,hamilton,albert_park
3,4,max_verstappen,max_verstappen,imola
4,5,max_verstappen,hamilton,miami
5,6,max_verstappen,russell,catalunya
6,7,perez,max_verstappen,monaco
7,8,max_verstappen,max_verstappen,baku
8,9,max_verstappen,hamilton,villeneuve
9,10,sainz,hamilton,silverstone


Neural Network Classifier (podium)

In [322]:
df_ohe2.podium = df_ohe2.podium.map(lambda x: x if x <= 3 else 0)

train = df_ohe2[df_ohe2.season < 2022]
test = df_ohe2[df_ohe2.season == 2022]
x_train = train.drop(['driver', 'podium', 'points'], axis = 1)
# x_test = test.drop(['driver', 'podium'], axis = 1)
y_train = train.podium
# y_test = test.podium

scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

In [325]:
final = []
def score_classification2(model):
    score = 0
    # final = []
    for circuit in df_ohe2[df_ohe2.season == 2022]['round'].unique():

        test = df_ohe2[(df_ohe2.season == 2022) & (df_ohe2['round'] == circuit)]
        test.reset_index(drop=True, inplace=True)

        # print(test.index)
        X_test = test.drop(['driver', 'podium', 'points'], axis = 1)
        y_test = test.podium
        # print(X_test)
        # print("ayyyyay")
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        # print(X_test)
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1', 'proba_2', 'proba_3'])
        prediction_df["actual"] = y_test
        prediction_df.reset_index(inplace = True, drop = True)
        
        prediction_df['predicted'] = prediction_df['proba_1']
        prediction_df['predicted2'] = prediction_df['proba_2']
        prediction_df['predicted3'] = prediction_df['proba_3']
        
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == max(prediction_df['proba_1'].values) else 0)
        prediction_df['predicted2'] = prediction_df.predicted2.map(lambda x: 2 if x == max(prediction_df['proba_2'].values) else 0)
        prediction_df['predicted3'] = prediction_df.predicted3.map(lambda x: 3 if x == max(prediction_df['proba_3'].values) else 0)
        
        second = prediction_df[prediction_df['predicted2'] == 2].index.values
        third = prediction_df[prediction_df['predicted3'] == 3].index.values
        
        prediction_df.loc[second, 'predicted'] = 2
        prediction_df.loc[third, 'predicted'] = 3
        
        prediction_df = prediction_df.drop(['predicted2', 'predicted3'], axis=1)
        
        # print(prediction_df)
        predWinner = prediction_df[prediction_df['predicted'] == 1].first_valid_index()
        predWinnerName = test['driver'].loc[predWinner]
        
        predSecond = prediction_df[prediction_df['predicted'] == 2].first_valid_index()
        predSecondName = test['driver'].loc[predSecond]
        
        predThird = prediction_df[prediction_df['predicted'] == 3].first_valid_index()
        predThirdName = test['driver'].loc[predThird]
        # winnerName = winner['driver']
        # print(winner)
        # actualWinner = test[test['podium'] == 1].first_valid_index
        actualWinnerName = test.loc[test['podium'] == 1, 'driver'].iloc[0]#test.loc[test['podium'] == 1, 'driver'].first_valid_index
        actualSecondName = test.loc[test['podium'] == 2, 'driver'].iloc[0]
        actualThirdName = test.loc[test['podium'] == 3, 'driver'].iloc[0]
        # print(actualWinnerName)
        final.append((circuit, [actualWinnerName, actualSecondName, actualThirdName], [predWinnerName, predSecondName, predThirdName]))
        
        score += precision_score(prediction_df.actual, prediction_df.predicted, average='macro')
    # print(final)
    model_score = score / df_ohe[df_ohe.season == 2022]['round'].unique().max()
    result = [model_score, prediction_df, final]
    return result

In [230]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

In [364]:
model2 = MLPClassifier(hidden_layer_sizes = (75, 25, 50, 10),
                                  activation = 'identity', solver = 'lbfgs', alpha = 0.01623776739188721, max_iter=2000, random_state = 1)

model2.fit(x_train, y_train)

MLPClassifier(activation='identity', alpha=0.01623776739188721,
              hidden_layer_sizes=(75, 25, 50, 10), max_iter=2000,
              random_state=1, solver='lbfgs')

In [317]:
model2 = MLPClassifier(hidden_layer_sizes = (80, 20, 40, 5),
                                  activation = 'tanh', solver = 'lbfgs', alpha = 100.0, max_iter=10000, random_state = 1)

model2.fit(x_train, y_train)

MLPClassifier(activation='tanh', alpha=100.0,
              hidden_layer_sizes=(80, 20, 40, 5), max_iter=10000,
              random_state=1, solver='lbfgs')

In [327]:
score_classification2(model2)   

KeyError: None

In [234]:
predictedDf = pd.DataFrame(final, columns=['CircuitId', 'Actual Podium', 'Predicted Podium'])
circuit_name = []
i = 1
for value in predictedDf['CircuitId']:
    circuit_name.insert(i, tempFinal.loc[(tempFinal['round'] == value) & (tempFinal['season'] == 2022), 'circuit_id'].iloc[0])
    i += 1
    
        

predictedDf['Circuit Name'] = circuit_name

In [235]:
predictedDf.tail()
display(predictedDf)

,CircuitId,Actual Podium,Predicted Podium,Circuit Name
0,1,"[leclerc, sainz, hamilton]","[leclerc, sainz, hamilton]",bahrain
1,2,"[max_verstappen, leclerc, sainz]","[max_verstappen, leclerc, sainz]",jeddah
2,3,"[leclerc, perez, russell]","[leclerc, perez, hamilton]",albert_park
3,4,"[leclerc, perez, max_verstappen]","[max_verstappen, perez, norris]",imola
4,5,"[max_verstappen, leclerc, sainz]","[max_verstappen, leclerc, sainz]",miami
5,6,"[leclerc, perez, russell]","[max_verstappen, perez, sainz]",catalunya
6,7,"[sainz, perez, russell]","[perez, sainz, max_verstappen]",monaco
7,8,"[max_verstappen, perez, russell]","[max_verstappen, perez, hamilton]",baku
8,9,"[max_verstappen, sainz, hamilton]","[max_verstappen, sainz, hamilton]",villeneuve
9,10,"[sainz, perez, hamilton]","[sainz, perez, hamilton]",silverstone


In [ ]:
# data = [['tom', 10], ['nick', 15], ['juli', 14]]
  
# # Create the pandas DataFrame
# df = pd.DataFrame(data, columns=['Name', 'Age'])
random 
prediction_df = pd.DataFrame(model.predict_proba(random), columns = ['proba_0', 'proba_1', 'proba_2', 'proba_3'])

SVM Classifier

In [328]:
df_ohe.podium = df_ohe.podium.map(lambda x: 1 if x == 1 else 0)



train = df_ohe[(df_ohe['season'] <= 2021) | ((df_ohe['season'] == 2022) & (df_ohe['round'] <= 11))]
test = df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] >= 12)]
# train = df_ohe[(df_ohe['season'] <= 2021)]
# test = df_ohe[(df_ohe['season'] == 2022)]
x_train = train.drop(['driver', 'podium', 'points'], axis = 1)
# x_test = test.drop(['driver', 'podium'], axis = 1)
y_train = train.podium
# y_test = test.podium

scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

In [329]:
# Support Vector Machines

params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            svmModel = svm.SVC(probability = True, gamma = gamma, C = c, kernel = kernel )
            svmModel.fit(x_train, y_train)
            
            model_score = score_classification(svmModel)
            
            comparison_dict['model'].append('svm_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [330]:
pd.DataFrame(comparison_dict).groupby('model')['params'].max()

model
neural_network_classifier    ((80, 20, 40, 5), tanh, lbfgs, 100.0)
svm_classifier                                (0.1, 10.0, sigmoid)
Name: params, dtype: object

In [331]:
model = svm.SVC(probability = True, gamma = 0.1, C = 10.0, kernel = 'sigmoid' )

In [332]:
model.fit(x_train, y_train)

SVC(C=10.0, gamma=0.1, kernel='sigmoid', probability=True)

In [337]:
print(score_classification(model))                                                   

(0.8181818181818182,      proba_0   proba_1  actual  predicted
0   0.687667  0.312333       1          1
1   0.856648  0.143352       0          0
2   0.824301  0.175699       0          0
3   0.886609  0.113391       0          0
4   0.984422  0.015578       0          0
5   0.964526  0.035474       0          0
6   0.981302  0.018698       0          0
7   0.991947  0.008053       0          0
8   0.992750  0.007250       0          0
9   0.990290  0.009710       0          0
10  0.991331  0.008669       0          0
11  0.992329  0.007671       0          0
12  0.992020  0.007980       0          0
13  0.991728  0.008272       0          0
14  0.996396  0.003604       0          0
15  0.993185  0.006815       0          0
16  0.992416  0.007584       0          0
17  0.987407  0.012593       0          0
18  0.994641  0.005359       0          0
19  0.988445  0.011555       0          0, [(12, 'max_verstappen', 'max_verstappen'), (13, 'max_verstappen', 'max_verstappen'), (14, 'max_v

In [357]:
df_ohe.podium = df_ohe.podium.map(lambda x: 1 if x == 1 else 0)



# train = df_ohe[(df_ohe['season'] <= 2021) | ((df_ohe['season'] == 2022) & (df_ohe['round'] <= 11))]
# test = df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] >= 12)]
train = df_ohe[(df_ohe['season'] <= 2021)]
test = df_ohe[(df_ohe['season'] == 2022)]
x_train = train.drop(['driver', 'podium', 'points'], axis = 1)
# x_test = test.drop(['driver', 'podium'], axis = 1)
y_train = train.podium
# y_test = test.podium

scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

In [358]:
#  & (df_ohe['round'] >= 12)
final = []
def SVMscore_classification(model):
    score = 0
    # final = []
    for circuit in df_ohe[(df_ohe.season == 2022)]['round'].unique():

        test = df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] == circuit)]
        test.reset_index(drop=True, inplace=True)

        # print(test.index)
        X_test = test.drop(['driver', 'podium', 'points'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df["actual"] = y_test
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df['proba_1']
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == max(prediction_df['proba_1'].values) else 0)
        predWinner = prediction_df[prediction_df['predicted'] == 1].first_valid_index()
        predWinnerName = test['driver'].loc[predWinner]
        # winnerName = winner['driver']
        # print(winner)
        # actualWinner = test[test['podium'] == 1].first_valid_index
        actualWinnerName = test.loc[test['podium'] == 1, 'driver'].iloc[0]#test.loc[test['podium'] == 1, 'driver'].first_valid_index
        # print(actualWinnerName)
        final.append((circuit, actualWinnerName, predWinnerName))
        # print(precision_score(prediction_df.actual, prediction_df.predicted))
        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df_ohe[(df_ohe.season == 2022) & (df_ohe['round'] >= 12)]['round'].unique().max()
    return model_score, prediction_df, final

In [360]:
params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            svmModel = svm.SVC(probability = True, gamma = gamma, C = c, kernel = kernel )
            svmModel.fit(x_train, y_train)
            
            model_score = SVMscore_classification(svmModel)
            
            comparison_dict['model'].append('svm_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [359]:
comparison_dict

{'model': [], 'params': [], 'score': []}

In [361]:
pd.DataFrame(comparison_dict).groupby('model')['params'].max()

model
svm_classifier    (0.1, 10.0, sigmoid)
Name: params, dtype: object

In [353]:
model = svm.SVC(probability = True, gamma = 0.1, C = 10.0, kernel = 'sigmoid' )

In [362]:
model.fit(x_train, y_train)

SVC(C=10.0, gamma=0.1, kernel='sigmoid', probability=True)

In [363]:
print(score_classification(model))     

(0.8181818181818182,      proba_0   proba_1  actual  predicted
0   0.701051  0.298949       1          1
1   0.814004  0.185996       0          0
2   0.794591  0.205409       0          0
3   0.840547  0.159453       0          0
4   0.966890  0.033110       0          0
5   0.942247  0.057753       0          0
6   0.974799  0.025201       0          0
7   0.990570  0.009430       0          0
8   0.992709  0.007291       0          0
9   0.986732  0.013268       0          0
10  0.990430  0.009570       0          0
11  0.991568  0.008432       0          0
12  0.991116  0.008884       0          0
13  0.990301  0.009699       0          0
14  0.995802  0.004198       0          0
15  0.991509  0.008491       0          0
16  0.990685  0.009315       0          0
17  0.970859  0.029141       0          0
18  0.995019  0.004981       0          0
19  0.984323  0.015677       0          0, [(1, 'leclerc', 'hamilton'), (2, 'max_verstappen', 'russell'), (3, 'leclerc', 'hamilton'), (4, '

In [387]:
predictedDf = pd.DataFrame(final, columns=['CircuitId', 'Actual Podium', 'Predicted Podium'])
circuit_name = []
i = 1
for value in predictedDf['CircuitId']:
    circuit_name.insert(i, tempFinal.loc[(tempFinal['round'] == value) & (tempFinal['season'] == 2022), 'circuit_id'].iloc[0])
    i += 1
    
        

predictedDf['Circuit Name'] = circuit_name

In [388]:
display(predictedDf)

,CircuitId,Actual Podium,Predicted Podium,Circuit Name
0,1,leclerc,hamilton,bahrain
1,2,max_verstappen,russell,jeddah
2,3,leclerc,hamilton,albert_park
3,4,max_verstappen,max_verstappen,imola
4,5,max_verstappen,hamilton,miami
5,6,max_verstappen,russell,catalunya
6,7,perez,max_verstappen,monaco
7,8,max_verstappen,max_verstappen,baku
8,9,max_verstappen,hamilton,villeneuve
9,10,sainz,hamilton,silverstone
